# Techcore Experience

---

## frAPI Benchmark

In [19]:
import time
import glob
import base64
import asyncio
import numpy as np
import requests as req
from pathlib import Path
import matplotlib.pyplot as plt

### Checking connection status from frAPI

In [18]:
resp = req.get('http://35.237.84.73/frapi/status')
print(resp)

<Response [200]>


---
#  Defining functions

In [4]:
def get_base64(image_path):
    with open(image_path, 'rb') as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')

In [5]:
def image_num(id):
    num = int(id)
    if num < 10:
        return f'000{num}'
    elif num < 100:
        return f'00{num}'
    else:
        return num

In [6]:
def img_name(name, id):
    return f'{name}_{image_num(id)}.jpg'

In [7]:
def img_path_pair(pair):
    first_name = img_name(pair[0], pair[1])
    second_name = img_name(pair[0], pair[2])
    
    first_path = Path(Path('dataset'), pair[0], first_name)
    second_path = Path(Path('dataset'), pair[0], second_name)
    
    return first_path, second_path

In [20]:
def get_matched_pairs_similarity_scores(matched_pairs):
    scores = []
    
    print('Loading...')
    
    for pair in matched_pairs:
        first_path, second_path = img_path_pair(pair)

        req_body = {
            'image_b641': get_base64(first_path),
            'image_b642': get_base64(second_path),
        }

        resp = req.post('http://35.237.84.73/frapi/verify/images', json = req_body)
        scores.append(resp.json()['similarity_score'])
    
    print('Done...')
    return scores

### Loading arrays

In [9]:
matched_pairs = np.loadtxt(Path('matched_pairs.txt'), dtype='str')
mismatched_pairs = np.loadtxt(Path('mismatched_pairs.txt'), dtype='str')

It takes around 5min to complete all 300 requests and populate ***matched_pairs_scores*** array

In [23]:
start = time.time()
matched_pairs_scores = get_matched_pairs_similarity_scores(matched_pairs)
elapsed = time.time() - start

print(f'Elapsed time: {int(elapsed)}s')
print('Score array length: ', len(matched_pairs_scores))

Loading...
Done...
Elapsed time: 277s
Score array length:  300


In [ ]:
for score in matched_pairs_scores:
    if score < 50 and